<a href="https://colab.research.google.com/github/alokchoudharyguliya/NLP/blob/main/Neural_Machine_Translation_with_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

In [1]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2025-08-18 18:23:23--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8143096 (7.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.77M  27.1MB/s    in 0.3s    

2025-08-18 18:23:24 (27.1 MB/s) - ‘fra-eng.zip’ saved [8143096/8143096]



In [2]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [5]:
text_dataset=tf.data.TextLineDataset("/content/dataset/fra.txt")

In [6]:
VOCAB_SIZE=20000
ENGLISH_SEQUENCE_LENGTH=64
FRENCH_SEQUENCE_LENGTH=64
EMBEDDING_DIM=300
BATCH_SIZE=64

In [ ]:
english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

In [ ]:
french_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=FRENCH_SEQUENCE_LENGTH
)

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,units):
    super(Encoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.units=units

  def build(self,input_shape):
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)
    self.lstm=LSTM(self.units,return_sequences=True)

  def call(self,x):
    x=self.embedding(x)
    output=self.lstm(x)
    return output

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self,units):
    super(BahdanauAttention,self).__init__()
    self.units=units

  def build(self,input_shape):
    self.w_1=tf.keras.layers.Dense(self.units)
    self.w_2=tf.keras.layers.Dense(self.units)
    self.w=tf.keras.layers.Dense(1)

  def call(self,prev_dec_state,enc_states):
    scores=self.w(
        tf.nn.tanh(
        self.w_1(tf.expand_dims(prev_dec_state,-2)) +
        self.w_2(enc_states)))

    attention_weights=tf.nn.softmax(scores,axis=1)
    context_vector=attention_weights*enc_states
    context_vector=tf.reduce_sum(context_vector,axis=1)
    return context_vector,attention_weights

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,dec_units,sequence_length):
    super(Decoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.dec_units=dec_units
    self.sequence_length=sequence_length

  def build(self,input_shape):
    self.dense=Dense(self.vocab_size,activation="softmax")
    self.gru=GRU(
        self.dec_units,return_sequences=True,return_state=True)
    self.attention=BahdanauAttention(self.dec_units)
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)

  def call(self,x,hidden,shifted_target):
    outputs=[]
    context_vectors=[]
    attention_weightss=[]
    shifted_target=self.embedding(shifted_target)

    for t in range(0,self.sequence_length):
      context_vector,attention_weights=self.attention(hidden,x)
      dec_input=context_vector+shifted_target[:,t]
      output,hidden=self.gru(tf.expand_dims(dec_input,1))
      outputs.append(output[:,0])

    outputs=tf.convert_to_tensor(outputs)
    outputs=tf.transpose(outputs, perm=[1,0,2])

    outputs=self.dense(outputs)
    return outputs,attention_weights

In [ ]:
### ENCODER
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
encoder=Encoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS)
encoder_output=encoder(input)

### DECODER
shifted_target=Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,FRENCH_SEQUENCE_LENGTH)
decoder_output,attention_weightss=decoder(encoder_output,tf.zeros([1,HIDDEN_UNITS]),shifted_target)

### OUTPUT
bahdanau=Model([input,shifted_target],decoder_output)
bahdanau.summary()